In [ ]:
import pandas as pd
import open3d as o3d
import numpy as np
import csv
import copy
import scipy
import os

In [ ]:
ZED_data = '../../data/ZED/720/point_cloud_PLY_3029_720_06-12-2022-19-39-57.ply'
os.path.exists(ZED_data)

In [ ]:
def appendSpherical_np(xyz):
    ptsnew = np.hstack((xyz, np.zeros(xyz.shape)))
    xy = xyz[:,0]**2 + xyz[:,1]**2
    ptsnew[:,3] = np.sqrt(xy + xyz[:,2]**2)
    # ptsnew[:,4] = np.arctan2(np.sqrt(xy), xyz[:,2]) # for elevation angle defined from Z-axis down
    ptsnew[:,4] = np.arctan2(xyz[:,2], np.sqrt(xy)) # for elevation angle defined from XY-plane up
    ptsnew[:,5] = np.arctan2(xyz[:,1], xyz[:,0])
    return ptsnew

In [ ]:
def back_to_pts_form(arr):
    # get the shape of the input array
    m, n = arr.shape
    azimuth_const = 90/n
    polar_const = 60/m
    
    # create a 3D output array of size (m * n, 3)
    out = np.zeros((m * n, 3))
    
    # populate the output array
    for row in range(m):
        for col in range(n):
            index = row * n + col
            out[index, 0] = arr[row, col]
            out[index, 1] = row * polar_const
            out[index, 2] = col * azimuth_const 
    
    return out

In [ ]:
pcd_zed = o3d.io.read_point_cloud(ZED_data)
pts_zed = np.asarray(pcd_zed.points)

In [ ]:
pts_zed_sph = appendSpherical_np(pts_zed)[:,3:6]
pts_zed_sph[:,1:3] = np.degrees(pts_zed_sph[:,1:3])

In [ ]:
a = pts_zed_sph[:,1]

min, max = pts_zed_sph[:,1].min() + 12, pts_zed_sph[:,1].max() - 12

b = pts_zed_sph[a > min]
pts_zed_sph_filt = b[b[:,1] < max]

print(pts_zed_sph)
print(pts_zed_sph_filt)

In [ ]:
v_const = 720/30
h_const = 1280/85

pts_zed_sph_filt[:,1] *= v_const
pts_zed_sph_filt[:,2] *= h_const  

np.savetxt("zed_filt.csv", pts_zed_sph_filt,delimiter=',')

In [ ]:
azimuth = (pts_zed_sph_filt[:,2].min(), pts_zed_sph_filt[:,2].max()) - pts_zed_sph_filt[:,2].min()
azimuth

In [ ]:
v_range = pts_zed_sph[:,1].min() + 12, pts_zed_sph[:,1].max() - 12 
h_range = pts_zed_sph[:,2].min(), pts_zed_sph[:,2].max()

# Split the array based on the range of the first column
arrays = []
for i in range(v_range):
    lower = i
    upper = i + 1
    mask = (pts_zed_sph[:, 0] >= lower) & (pts_zed_sph[:, 0] < upper)
    sub_array = pts_zed_sph[mask, 1:]
    arrays.append(sub_array)


In [ ]:

# Convert each sub-array to a 1D array sorted by the first column and using the second column as the value
sph_lidar_frame = []
for sub_array in arrays:
    indices = np.argsort(sub_array[:, 0])
    sorted_array = sub_array[indices, 1].tolist()

    row_len = len(sorted_array)
    num_zeros = 1280 - row_len
    step_size = row_len // (num_zeros + 1)

    # Loop over the array and insert zeros at regular intervals
    for i in range(num_zeros):
        index = (i + 1) * step_size
        sorted_array.insert(index, 0)
    print(len(sorted_array))
    
    sph_lidar_frame.append(sorted_array)

In [ ]:
back_pts_zed = back_to_pts_form()

In [ ]:
o3d.visualization.draw_geometries([pcd_zed])